In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import os

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets('/mnist',one_hot=True)

Extracting /mnist\train-images-idx3-ubyte.gz
Extracting /mnist\train-labels-idx1-ubyte.gz
Extracting /mnist\t10k-images-idx3-ubyte.gz
Extracting /mnist\t10k-labels-idx1-ubyte.gz


In [5]:
n_samples = mnist.train.num_examples
batchsize = 100

## Encoder

In [6]:
x = tf.placeholder(tf.float32,[batchsize,784],name = "inputs")
x_image = tf.reshape(x,[batchsize,28,28,1],name ="imagereshape")

In [7]:
def init_weights(shape,n):
    w_vals = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(w_vals,name = n +'Weight')

In [8]:
def init_bias(shape,n):
    b_vals  = tf.constant(0.1,shape = shape)
    return tf.Variable(b_vals,name=n+'Bias')

In [9]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides = [1,2,2,1],padding='SAME')

In [10]:
def convolutional_layer(x,shape,name,namescope):
    with tf.name_scope(namescope):
        W = init_weights(shape,name)
        b = init_bias([shape[3]],n=name)
        return  tf.nn.relu(conv2d(x,W) + b)

In [11]:
conv1 = convolutional_layer(x_image,[6,6,1,16],name = "ConvL1",namescope = "Layer1") ### output 16 feature maps (28*28*16)/2 == 14*14*16

In [12]:
conv2 = convolutional_layer(conv1,shape = [6,6,16,32],name = "ConvL2",namescope= "Layer2") ## ==7*7*32

In [13]:
convo_2_flat = tf.reshape(conv2,[batchsize,7*7*32])

In [14]:
def fully_connected_layer(input_layer,size,name,namescope):
    with tf.name_scope(namescope):
        input_size = int(input_layer.get_shape()[1])
        W = init_weights([input_size,size],name)
        b = init_bias([size],n=name)
        return tf.nn.relu(tf.matmul(input_layer,W)+b)

In [15]:
z_mean = fully_connected_layer(convo_2_flat,20,"FCL",namescope="Layer3")
z_stddev = fully_connected_layer(convo_2_flat,20,"FCL",namescope="Layer4")

In [16]:
z_mean

<tf.Tensor 'Layer3/Relu:0' shape=(100, 20) dtype=float32>

In [17]:
samples = tf.random_normal([batchsize,20],0,1,dtype=tf.float32)

In [18]:
samples

<tf.Tensor 'random_normal:0' shape=(100, 20) dtype=float32>

In [19]:
guessed_z = z_mean + (samples)*(z_stddev)

In [20]:
guessed_z

<tf.Tensor 'add:0' shape=(100, 20) dtype=float32>

## Decoder

In [21]:
z_develop = fully_connected_layer(guessed_z,7*7*32,"DecoderFCL","Layer1")

In [22]:
z_develop

<tf.Tensor 'Layer1_1/Relu:0' shape=(100, 1568) dtype=float32>

In [23]:
z_matrix = tf.reshape(z_develop,[batchsize,7,7,32],name ="Reshape")

In [24]:
z_matrix

<tf.Tensor 'Reshape_2:0' shape=(100, 7, 7, 32) dtype=float32>

In [25]:
def conv2d_transpose(x,W,output_shape):
    return tf.nn.conv2d_transpose(x,W,output_shape=output_shape,strides =[1,2,2,1])

In [26]:
def deconv(x,output_shape,shape,name,namescope):
    with tf.name_scope(namescope):
        W = init_weights(shape,name)
        b = init_bias([shape[3]],n=name)
        return tf.nn.relu(conv2d_transpose(x,W,output_shape))

In [27]:
deconv_2 = deconv(z_matrix,[batchsize,14,14,16],[5,5,16,32],"DeconvL1","Layer2")

In [28]:
deconv_2

<tf.Tensor 'Layer2_1/Relu:0' shape=(100, 14, 14, 16) dtype=float32>

In [29]:
deconv_1 = deconv(deconv_2,[batchsize,28,28,1],[5,5,1,16],"DeconvL2","Layer3")

In [30]:
generated_flat = tf.reshape(deconv_1,[batchsize,28*28],name ="Flattening")

In [31]:
x

<tf.Tensor 'inputs:0' shape=(100, 784) dtype=float32>

## Loss function And Optimiser

In [32]:
decoder_loss = -tf.reduce_sum(x*tf.log(1e-8 + generated_flat) + (1-x)*tf.log(1e-8 + 1 - generated_flat),1,name ="Decode_loss")

In [33]:
latent_loss = 0.5*tf.reduce_sum(tf.square(z_mean)+tf.square(z_stddev) - tf.log(tf.square(z_stddev))-1,1,name ="Latent_loss")

In [34]:
cost = tf.reduce_mean(decoder_loss + latent_loss,name ="Cost")

In [35]:
optimizer = tf.train.AdamOptimizer(0.00001).minimize(cost,name ="optimizer")

In [36]:
### gpu settings
#config = tf.ConfigProto()

In [37]:
#config.gpu_options.allow_growth = True

In [38]:
#session = tf.Session(config=config)

In [39]:
init = tf.global_variables_initializer()

In [40]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("./output",sess.graph)
    batch = mnist.train.next_batch(batchsize)[0]
    sess.run(optimizer, feed_dict={x: batch})
    writer.close()

In [ ]:
no_of_epoches = 10
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(no_of_epoches):
        for idx in range(int(n_samples/batchsize)):
            batch = mnist.train.next_batch(batchsize)[0]
            _, gen_loss, lat_loss = sess.run((optimizer,decoder_loss, latent_loss), feed_dict={x: batch})
            if idx%(n_samples-3)==0:
                print ("epoch %d: genloss %f latloss %f" % (epoch, np.mean(gen_loss), np.mean(lat_loss)))